## TTS

In [1]:
import os

import torch
import gradio as gr
from TTS.api import TTS
os.environ["COQUI_TOS_AGREED"] = "1"
os.environ["PYTHONPATH"] = "/usr/lib/python3.10/dist-packages/"

from kitt.core import voices, tts_gradio

/home/ubix/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
speaker_embedding_cache = {}

In [3]:
out = tts_gradio("The path to glory isn't paved. Be very careful what you say", "Freeman - Neutral", speaker_embedding_cache)

2024-05-21 14:20:22.300 | INFO     | kitt.core:load_tts_pipeline:58 - Loading model tts_models/multilingual/multi-dataset/xtts_v2 into device: cuda
/home/ubix/TTS/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.


ModuleNotFoundError: No module named 'tensorrt'

## STT


In [8]:
import torch
from transformers import pipeline


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
transcriber = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base.en", device=device
)

/home/ubix/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
out[0]

(22050,
 array([-0.00043467, -0.00379933, -0.00812101, ...,  0.        ,
         0.        ,  0.        ]))

In [15]:
sr, y = out[0]

In [18]:
%timeit transcriber({"sampling_rate": sr, "raw": y})

391 ms ± 39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
text

{'text': ' The path to glory is in paves. Be very careful what you say.'}

## LLM

In [1]:
from langchain.memory import ChatMessageHistory
from langchain.tools.base import StructuredTool


from kitt.core.model import process_query
from kitt.skills import (
    get_weather,
    find_route,
    get_forecast,
    vehicle_status as vehicle_status_fn,
    search_points_of_interests,
    search_along_route_w_coordinates,
    do_anything_else,
    date_time_info,
    get_weather_current_location,
    code_interpreter,
)

In [2]:
history = ChatMessageHistory()
tools = [
    StructuredTool.from_function(get_weather),
    StructuredTool.from_function(find_route),
    StructuredTool.from_function(vehicle_status_fn),
    StructuredTool.from_function(search_points_of_interests),
    # StructuredTool.from_function(search_along_route),
    StructuredTool.from_function(date_time_info),
    StructuredTool.from_function(get_weather_current_location),
    StructuredTool.from_function(code_interpreter),
    # StructuredTool.from_function(do_anything_else),
]

In [3]:
process_query("How is the weather?", history, tools)

Prompt is:<|im_start|>system
You are a function calling AI agent with self-recursion.
You can call only one function at a time and analyse data you get from function response.
You are provided with function signatures within <tools></tools> XML tags.

You may use agentic frameworks for reasoning and planning to help with user query.
Please call a function and wait for function results to be provided to you in the next iteration.
Don't make assumptions about what values to plug into function arguments.
Once you have called a function, results will be fed back to you within <tool_response></tool_response> XML tags.
Don't make assumptions about tool results if <tool_response> XML tags are not present since function hasn't been executed yet.
Analyze the data once you get the results and call another function.
At each iteration please continue adding the your analysis to previous summary.
Your final response should directly answer the user query. Don't tell what you are doing, just do it.



ResponseError: model 'dolphin-llama3:8b' not found, try pulling it first